In [33]:
!python mmdetection/tools/train.py CascadeTabNet/Config/new_config.py

Training Started
08/13 16:42:03 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1634574429
    GPU 0: NVIDIA A30
    CUDA_HOME: None
    GCC: gcc (GCC) 12.3.0
    PyTorch: 2.3.1
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.6 (Git Hash 86e6af5974177e513fd3fee58425e1063e7f1361)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=comput

In [1]:
import torch
print(torch.cuda.is_available())

True


In [2]:
!pwd

/csehome/m23csa016/MTP


In [12]:
!python mmdetection/tools/test.py CascadeTabNet/Config/new_config.py CascadeTabNet/Checkpoints/Original/new_config/epoch_100.pth --work-dir CascadeTabNet/evaluations/Original/

08/19 14:57:19 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1223103740
    GPU 0: NVIDIA A30
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 12.2, V12.2.91
    GCC: gcc (GCC) 12.3.0
    PyTorch: 2.3.1
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.6 (Git Hash 86e6af5974177e513fd3fee58425e1063e7f1361)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-genc

In [4]:
%cd MTP

/csehome/m23csa016/MTP


/csehome/m23csa016/.conda/envs/pyten/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
from mmdet.apis import DetInferencer

# Initialize the DetInferencer
inferencer = DetInferencer(model='CascadeTabNet/Config/new_config.py', weights='/csehome/m23csa016/MTP/work_dirs/new_config/epoch_64.pth')

# Perform inference
bbox = inferencer('/csehome/m23csa016/largepreview.png', out_dir='CascadeTabNet/testres', no_save_pred=False)

In [3]:
%cd ..

/csehome/m23csa016/MTP


/csehome/m23csa016/.conda/envs/pyten/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
import json
from mmdet.apis import init_detector, inference_detector
import mmcv
import os
from mmdet.apis import DetInferencer

# Paths
# Specify the path to model config and checkpoint file
data_root = "/scratch/m23csa016/tabdet_data"
config_path = 'CascadeTabNet/Config'
checkpoint_path = 'CascadeTabNet/Checkpoints'

image_folder = os.path.join(data_root, "Orig_Image")  # Single folder for all images
test_annotation_file = os.path.join(data_root, "Annotations/test.json")  # Test annotation file path
output_dir = 'CascadeTabNet/testres'

for p in [10, 15, 20, 25, 30, 40]:
    config_file = os.path.join(config_path, f"config_{p}.py")
    checkpoint_file = os.path.join(checkpoint_path, f"Data_{p}/train_{p}/epoch_40.pth")

    out_dir = os.path.join(output_dir, f'data_{p}')
    os.makedirs(out_dir, exist_ok=True)

    pred_dir = os.path.join(out_dir, "preds")
    os.makedirs(pred_dir, exist_ok=True)

    # Initialize the DetInferencer
    inferencer = DetInferencer(model=config_file, weights=checkpoint_file)
    # Load test annotations
    with open(test_annotation_file) as f:
        test_annotations = json.load(f)

    # Get test image filenames
    test_image_filenames = [img['file_name'] for img in test_annotations['images']]

    # Run inference and save results
    for img_filename in test_image_filenames:
        img_path = os.path.join(image_folder, img_filename)
        
        # Perform inference
        result = inferencer(img_path, out_dir=out_dir)

        # Process the result to the desired format
        output = []
        # Extracting labels, scores, and bboxes
        if len(result['predictions'] != 0):
            label = result['predictions'][0]['labels'][0] 
            score = result['predictions'][0]['scores'][0]
            bbox = result['predictions'][0]['bboxes'][0]
        else:
            label = 0
            score = 0
            bbox = [0, 0, 0, 0]
        
        x_min, y_min, x_max, y_max = bbox[0], bbox[1], bbox[2], bbox[3]

        output.append(f"{label} {score:.4f} {x_min:.2f} {y_min:.2f} {x_max:.2f} {y_max:.2f}")
        
        # Save the results to a file
        with open(f"{pred_dir}/{img_filename}.txt", 'w') as f:
            f.write("\n".join(output))


IndentationError: expected an indented block after 'else' statement on line 50 (2352594352.py, line 52)

In [3]:
%cd ..

/csehome/m23csa016/MTP


/csehome/m23csa016/.conda/envs/pyten/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [8]:
!python mmdetection/tools/analysis_tools/analyze_logs.py plot_curve CascadeTabNet/work_dirs/new_config/20240816_192104/vis_data/20240816_192104.json --keys s0.acc s1.acc s2.acc --legend s0.acc s1.acc s2.acc --out CascadeTabNet/evaluations/Dilated/plots/accuracy.pdf

plot curve of CascadeTabNet/work_dirs/new_config/20240816_192104/vis_data/20240816_192104.json, metric is s0.acc
plot curve of CascadeTabNet/work_dirs/new_config/20240816_192104/vis_data/20240816_192104.json, metric is s1.acc
plot curve of CascadeTabNet/work_dirs/new_config/20240816_192104/vis_data/20240816_192104.json, metric is s2.acc
save curve to: CascadeTabNet/evaluations/Dilated/plots/accuracy.pdf


In [ ]:
import os

def remove_prefix_from_filenames(directory, prefix):
    for filename in os.listdir(directory):
        if filename.startswith(prefix):
            new_filename = filename[len(prefix):]
            old_file_path = os.path.join(directory, filename)
            new_file_path = os.path.join(directory, new_filename)
            os.rename(old_file_path, new_file_path)
            print(f'Renamed: {old_file_path} -> {new_file_path}')

# Example usage
directory = '/scratch/m23csa016/tabdet_data/Smudged'
prefix = 'Smudge_'
remove_prefix_from_filenames(directory, prefix)


In [15]:
import json
import os
import random

def create_subset_json(input_json_path, output_json_path, subset_percentage):
    # Load the original JSON file
    with open(input_json_path, 'r') as f:
        data = json.load(f)
    
    # Calculate the number of images and annotations to include in the subset
    total_images = len(data['images'])
    subset_size = int(total_images * subset_percentage / 100)
    
    # Randomly select a subset of images
    subset_images = random.sample(data['images'], subset_size)
    subset_image_ids = set([image['id'] for image in subset_images])
    
    # Filter annotations that correspond to the selected images
    subset_annotations = [anno for anno in data['annotations'] if anno['image_id'] in subset_image_ids]
    
    # Create the subset JSON structure
    subset_data = {
        'images': subset_images,
        'annotations': subset_annotations,
        'categories': data['categories']
    }
    
    # Save the subset JSON to a new file
    with open(output_json_path, 'w') as f:
        json.dump(subset_data, f, indent=4)

# Example usage:

# create_subset('train.json', 'train_15.json', 15)
# create_subset('train.json', 'train_20.json', 20)

Subset JSON file created: train_10.json


In [17]:
data_root = "/scratch/m23csa016/tabdet_data/Annotations/"

In [ ]:
percentages = [10, 15, 20, 25, 30, 40]

for per in percentages:
    create_subset_json(os.path.join(data_root, 'train.json'), os.path.join(data_root, f'train_{per}.json'), per)   

In [22]:
import os
import json

def update_config(config_path, percentage, subset_json_path, checkpoint_dir):
    with open(config_path, 'r') as file:
        config_data = file.read()

    config_data = config_data.replace("Annotations/train.json", f'Annotations/train_{percentage}.json')
    
    # Update the checkpoint file path
    checkpoint_path = os.path.join(checkpoint_dir, f'Data_{percentage}')
    os.makedirs(checkpoint_path, exist_ok=True)

    config_data = config_data.replace("out_dir='/csehome/m23csa016/MTP/CascadeTabNet/Checkpoints/Orig_Image'", f"out_dir='{checkpoint_path}'")
    
    # Write the updated config to a new file
    new_config_path = os.path.join("/csehome/m23csa016/MTP/CascadeTabNet/Config", f'config_{percentage}.py')
    with open(new_config_path, 'w') as file:
        file.write(config_data)
    
    return new_config_path

config_path = 'CascadeTabNet/Config/new_config.py'
subset_json_path = '/scratch/m23csa016/tabdet_data/Annotations'
checkpoint_dir = '/csehome/m23csa016/MTP/CascadeTabNet/Checkpoints'

for percentage in [10, 15, 20, 25, 30, 40]:
    new_config_path = update_config(config_path, percentage, subset_json_path, checkpoint_dir)

In [21]:
work_dir = "/csehome/m23csa016/MTP/CascadeTabNet/work_dirs"

for p in [10, 15, 20, 25, 30, 40]:
    os.makedirs(os.path.join(work_dir, f'train_{p}'))